<a href="https://colab.research.google.com/github/konkolyseismolab/seismolab/blob/master/docs/source/query_gaia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

[Open this tutorial in Google Colab](https://colab.research.google.com/github/konkolyseismolab/seismolab/blob/master/docs/source/query_gaia.ipynb)

# Query Gaia catalog

The *Gaia* module combines Gaia data with the Bailer-Jones distance catalog, galactic extinction maps and magnitudes from the Simbad catalog. Simply passing the Gaia ID of one or more targets, the code automatically queries

- the Gaia parallax,
- the Gaia G, BP and RP brightness,
- the Johnson B, V and 2MASS JHK brightness,

and delivers

- the extinction corrected apparent brightness,
- the extinction corrected absolute brightness,
- extinction values in each band,

with the associated upper and lower errorbars.

__Note:__
Both Gaia DR2 and DR3 catalogs can be used. However, there is a subtle difference between them. In case of DR3, the distance are queried from the [Bailer-Jones catalog](https://ui.adsabs.harvard.edu/abs/2021AJ....161..147B/abstract). If DR2 is used, the distances are estimated from the Gaia parallaxes in a probabilistic way as defined by [Bailer-Jones+ (2018, AJ, 156, 58)](https://ui.adsabs.harvard.edu/abs/2018AJ....156...58B/abstract).

## __How to use__

In [1]:
from seismolab.gaia import query_gaia

After importing the relevant module, we can pass a list of __Gaia DR3 IDs__ to perform the query and all the calculations.

In [2]:
targets = [1022096392749670016,3107142829863485952]

result_table = query_gaia(targets)

Using Combined19 map from mwdust
Querying Gaia DR3 and BJ catalogs...
Calculating distances, absolute magnitudes...


  0%|          | 0/2 [00:00<?, ?it/s]

The result is an Astropy Table.

In [3]:
result_table

<Table length=2>
       Source               ra         ...         mKem        
                           deg         ...                     
       str21             float64       ...       float64       
------------------- ------------------ ... --------------------
1022096392749670016 145.93959651092428 ... 0.017000198364257812
3107142829863485952 101.76822041702867 ... 0.054260166043041735

The columns are named according to the followings:

- __Source:__ Gaia DR3 (or DR2) ID 
- __ra, dec:__ equatorial coordinates in deg
- __parallax, sig_parallax:__ parallax and its error in mas
- __\<filter\>mag:__ observed magnitude in given filter
- __sig_\<filter\>mag:__ observed magnitude error in given filter
- __dist:__ distance in pc
- __distep:__ upper distance errorbar in pc
- __distem:__ lower distance errorbar in pc
- __lon_deg, lat_deg:__ galactic coordinates in deg
- __a\<filter>:__ extinction in given filter
- __a\<filter>ep:__ upper extinction errorbar in given filter
- __a\<filter>em:__ lower extinction errorbar in given filter
- __M\<filter>:__ extinction corrected absolute magnitude in given filter
- __M\<filter>ep:__ upper absolute magnitude errorbar in given filter
- __M\<filter>em:__ lower absolute magnitude errorbar in given filter
- __m\<filter>:__ extinction corrected apparent magnitude in given filter
- __m\<filter>ep:__ upper apparent magnitude errorbar in given filter
- __m\<filter>em:__ lower apparent magnitude errorbar in given filter

### Using photodistances

The [Bailer-Jones catalog](https://ui.adsabs.harvard.edu/abs/2021AJ....161..147B/abstract) contains two kinds of distances. By default, the geometric ones are returned, but there is an option to use the __photodistances__ instead.

In [4]:
result_table = query_gaia(targets,use_photodist=True)

Using Combined19 map from mwdust
Querying Gaia DR3 and BJ catalogs...
Calculating distances, absolute magnitudes...


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
result_table

<Table length=2>
       Source               ra         ...         mKem        
                           deg         ...                     
       str21             float64       ...       float64       
------------------- ------------------ ... --------------------
1022096392749670016 145.93959651092428 ... 0.017000198364257812
3107142829863485952 101.76822041702867 ... 0.047101951442849455

### Querying DR2 catalog

If DR2 catalog is queried, then __Gaia DR2 IDs__ must be passed.

In [6]:
result_table_dr2 = query_gaia(targets,gaiaDR=2)

Using Combined19 map from mwdust
Querying Gaia DR2 catalog...
Calculating distances, absolute magnitudes...


  0%|          | 0/2 [00:00<?, ?it/s]

The result is an Astropy Table with distance probabilistically estimated from parallaxes.

In [7]:
result_table_dr2

<Table length=2>
       Source               ra         ...         mKem        
                           deg         ...                     
       str21             float64       ...       float64       
------------------- ------------------ ... --------------------
1022096392749670016 145.93959226288555 ... 0.016895750209253713
3107142829863485952 101.76822078135328 ...  0.04095751794617364

### Extinction correction

The extinction correction is done using the [mwdust](https://github.com/jobovy/mwdust) galactic dust maps. Given the coordinates and the distance of a target, `mwdust` returns the $E(B-V)$ color excess. From this, the $A_{\lambda}$ extinction coefficient can be calculated using the extinction vector $R_{\lambda}$:
$$ A_{\lambda} = R_{\lambda} \times E(B-V) ,$$
where $\lambda$ denotes the passband.

The values of the extinction vector are from [Green+ (2019)](https://ui.adsabs.harvard.edu/abs/2019ApJ...887...93G/abstract), whose results are based on [Schlafly+ (2016)](https://ui.adsabs.harvard.edu/abs/2016ApJ...821...78S/abstract). Since not all filters are available, [Huber+ (2017)](https://ui.adsabs.harvard.edu/abs/2017ApJ...844..102H/abstract) interpolated [Cardelli+ (1989)](https://ui.adsabs.harvard.edu/abs/1989ApJ...345..245C/abstract) extinction law to get extinction in e.g., the Gaia passbands, too.

| Passband | $R_{\lambda}$ |
| --- | --- |
| Johnson-Cousins B | 3.8681 |
| Johnson-Cousins V | 3.0576 |
| Tycho B$_{\rm T}$ | 3.9958 |
| Tycho V$_{\rm T}$  | 3.1870 |
| SDSS u | 4.5890 |
| SDSS g | 3.6134 |
| SDSS r | 2.6198 |
| SDSS i | 1.9872 |
| SDSS z | 1.4617 |
| 2MASS J | 0.7927 |
| 2MASS H | 0.4690 |
| 2MASS K | 0.3026 |
| Gaia G | 2.8135 |
| Gaia BP | 3.3609 |
| Gaia RP | 1.9427 |

#### Selecting the dust map

There are several dust maps implemented in `mwdust` (see [here](https://github.com/jobovy/mwdust?tab=readme-ov-file#dust-data)). By default the _Combined19_ map is used, however any other available map can also be selected. E.g. changing the map to Green19 can be done by passing the name of the map to `dustmodel`.

__Note:__ the desired dust map _might_ be downloaded beforehand.

__Note:__ if the distance of the star is too large and the color excess falls outside the reliable range of the given dust map, no warning is given!

In [8]:
result_table = query_gaia(targets,dustmodel='Green19')

Using Green19 map from mwdust
Querying Gaia DR3 and BJ catalogs...
Calculating distances, absolute magnitudes...


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
result_table

<Table length=2>
       Source               ra         ...         mKem        
                           deg         ...                     
       str21             float64       ...       float64       
------------------- ------------------ ... --------------------
1022096392749670016 145.93959651092428 ... 0.017000198364257812
3107142829863485952 101.76822041702867 ... 0.054260166043041735

### Setting the parallax offset in Gaia DR2

A parallax offset can be given if the Gaia DR2 catalog is used. There are four predefined options, which can be selected:

- __Stassun__ : +0.08 mas (Stassun et al. 2018)
- __Riess__ : +0.046  mas (Riess et al. 2018)
- __BJ__ : +0.029  mas (Bailer-Jones et al. 2018)
- __Zinn__ : +0.0528 mas (Zinn et al 2019)
        
or simply an offset in _mas_ can be passed.

In [10]:
result_table = query_gaia(targets,gaiaDR=2,plx_offset='Stassun')

Using Combined19 map from mwdust
Querying Gaia DR2 catalog...
Calculating distances, absolute magnitudes...


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
result_table

<Table length=2>
       Source               ra         ...         mKem        
                           deg         ...                     
       str21             float64       ...       float64       
------------------- ------------------ ... --------------------
1022096392749670016 145.93959226288555 ... 0.016895750209253713
3107142829863485952 101.76822078135328 ...  0.03456370625147187